This notebook requires three dataframes:
* competences_labels.xlsx
* correctNormalization.xlsx
* furthered_modified_data_set_demo_accounts_removed (3)

In [ ]:
import pandas as pd
df = pd.read_excel("furthered_modified_data_set_demo_accounts_removed (3).xlsx")

#Inspect dataset
print(df.head(5))

   question_id  dossier_id answer answeredTest_id          question  \
0         2281        1152    0.6           19833  question.id_2281   
1         2263        1152    0.6           19833  question.id_2263   
2         2269        1152    0.6           19833  question.id_2269   
3         2278        1152    0.6           19833  question.id_2278   
4         2280        1152    0.6           19833  question.id_2280   

                                            type  \
0  likert/likert5puntsacceptatie2positiefpluseen   
1  likert/likert5puntsacceptatie2positiefpluseen   
2  likert/likert5puntsacceptatie2positiefpluseen   
3  likert/likert5puntsacceptatie2positiefpluseen   
4  likert/likert5puntsacceptatie2positiefpluseen   

                                     label            datetime  \
0                      rapporten schrijven 2022-06-27 12:36:27   
1                           intuïtief zijn 2022-06-27 12:36:27   
2             je gevoelens duidelijk uiten 2022-06-27 12:36:27

In [ ]:
#Inspect dataset
norm_df = pd.read_excel("correctNormalization.xlsx")
norm_df

,dossier_id,question_id,answer,label
0,973,2336,1.0,Dat ik de hele dag bezig kan zijn
1,973,2355,0.8,Dat ik het gevoel heb dat ik iets bereik met w...
2,973,2354,0.8,Dat ik een vaste baan heb
3,973,2337,1.0,Dat ik elke dag iets anders kan doen
4,973,2338,0.6,Dat ik dingen voor anderen kan doen
...,...,...,...,...
119554,91930,2411,0.8,Om mijn werk goed te doen heb je creativiteit ...
119555,91930,2397,0.4,Om mijn werk goed te doen is het belangrijk om...
119556,91930,2398,0.6,Om mijn werk goed te doen is doorzettingsvermo...
119557,91930,2399,0.6,Om mijn werk goed te doen is het belangrijk om...


In [ ]:
#Merge both datasets
df = df.merge(norm_df[['dossier_id', 'question_id', 'answer']], on=['dossier_id', 'question_id'], suffixes=('_df', '_norm'))

In [ ]:
#Drop answer_df
df.drop(['answer_df'], axis=1, inplace=True)

#Rename 'answer_norm' dataset
df = df.rename(columns={'answer_norm': 'answer'})
print(df.head(5))
finaldf = df

   question_id  dossier_id answeredTest_id          question  \
0         2281        1152           19833  question.id_2281   
1         2263        1152           19833  question.id_2263   
2         2269        1152           19833  question.id_2269   
3         2278        1152           19833  question.id_2278   
4         2280        1152           19833  question.id_2280   

                                            type  \
0  likert/likert5puntsacceptatie2positiefpluseen   
1  likert/likert5puntsacceptatie2positiefpluseen   
2  likert/likert5puntsacceptatie2positiefpluseen   
3  likert/likert5puntsacceptatie2positiefpluseen   
4  likert/likert5puntsacceptatie2positiefpluseen   

                                     label            datetime  \
0                      rapporten schrijven 2022-06-27 12:36:27   
1                           intuïtief zijn 2022-06-27 12:36:27   
2             je gevoelens duidelijk uiten 2022-06-27 12:36:27   
3  activiteiten en evenementen organis

In [ ]:
#Print the unique values of level1subgroup1 for each unique maingroup
grouped = df.groupby('maingroup')['level1subgroup1'].unique()

for maingroup, subgroups in grouped.items():
    print("maingroup:", maingroup)
    print("level1subgroup1 values:", subgroups)
    print()



maingroup: Beroepskeuzetest
level1subgroup1 values: ['Conventioneel' 'Artistiek' 'Sociaal' 'Ondernemend' 'Realistisch'
 'Intellectueel']

maingroup: Competenties
level1subgroup1 values: ['\\N']

maingroup: tests.workstyles.title
level1subgroup1 values: ['Gerichtheid op mensen' 'Sociale invloed' 'Aanpassingsvermogen'
 'Zorgvuldigheid' 'Praktisch inzicht' 'Prestatiegerichtheid'
 'Onafhankelijkheid']

maingroup: tests.workvalues.title
level1subgroup1 values: ['Prestaties leveren' 'Arbeidsvoorwaarden' 'Erkenning' 'Collegialiteit'
 'Ondersteuning' 'Onafhankelijkheid']



##OBTAIN PERSONALITY TRAIT TEST RESULTS

In [ ]:
#Print range of answer values for personality trait questions
min_value = df[df['maingroup'] == 'Beroepskeuzetest']['answer'].min()
max_value = df[df['maingroup'] == 'Beroepskeuzetest']['answer'].max()

print("Minimum value:", min_value)
print("Maximum value:", max_value)

Minimum value: 0.0
Maximum value: 1.0


In [ ]:
import numpy as np

#Subset of the original DataFrame
subset_df = df[df['maingroup'] == 'Beroepskeuzetest']

#Create the 'personality_result' DataFrame with desired columns
personality_result = pd.DataFrame(columns=['dossier_id', 'Conventioneel', 'Artistiek', 'Sociaal', 'Ondernemend', 'Realistisch', 'Intellectueel'])

#Iterate over unique dossier_id values in the 'subset_df' and append the new row to 'personality_result'
for dossier_id in subset_df['dossier_id'].unique():
    new_row = pd.DataFrame({'dossier_id': [dossier_id],
                            'Conventioneel': [0],
                            'Artistiek': [0],
                            'Sociaal': [0],
                            'Ondernemend': [0],
                            'Realistisch': [0],
                            'Intellectueel': [0]})
    personality_result = personality_result.append(new_row, ignore_index=True)

#Iterate over each row in the 'subset_df'
for index, row in subset_df.iterrows():
    dossier_id = row['dossier_id']
    answer = row['answer']
    level1subgroup1 = row['level1subgroup1']

    #Calculate the difference between 0.4 and the answer value
    difference = 0.5 - float(answer)

    #Check if the difference is positive or negative and update the 'personality_result' using the difference
    if difference > 0:
        personality_result.loc[personality_result['dossier_id'] == dossier_id, level1subgroup1] -= difference
    elif difference < 0:
        personality_result.loc[personality_result['dossier_id'] == dossier_id, level1subgroup1] += abs(difference)

#Check the resulting dataset
print(personality_result)


In [ ]:
#Obtain the personality trait with the highest score for each dossierid

#List of the columns to consider
columns_to_check = ['Conventioneel', 'Artistiek', 'Sociaal', 'Ondernemend', 'Realistisch', 'Intellectueel']

#Create a new column 'pers_trait' and initialize it with None
personality_result['pers_trait'] = None

#Iterate over each row
for index, row in personality_result.iterrows():
    max_value = None
    max_column = None

    #Iterate over the columns to check
    for column in columns_to_check:
        value = row[column]

        #Update the maximum value and column if necessary
        if max_value is None or value > max_value:
            max_value = value
            max_column = column

    #Assign the name of the column with the highest value to 'pers_trait'
    personality_result.at[index, 'pers_trait'] = max_column

print(personality_result)


    dossier_id Conventioneel Artistiek Sociaal Ondernemend Realistisch  \
0         1152          3.25       4.5    3.25         2.0        -1.5   
1         1532          3.25       1.0    5.25        2.25        2.25   
2         1649         -1.25      3.75    3.75         0.5        -3.0   
3         1663         -3.75      4.75     6.0         2.5        0.75   
4         2112           1.5      2.75    6.25        7.25        0.25   
..         ...           ...       ...     ...         ...         ...   
744      91894           5.0      2.25    1.75         1.0       -0.75   
745      91908           1.5       0.0     3.0       -0.25         1.0   
746      91927          1.75      0.75    2.25        -1.5        -2.5   
747      91928          3.25      2.25     3.5         2.5        4.75   
748      91930          2.25       2.5    2.75        1.25       -0.75   

    Intellectueel     pers_trait  
0             1.5      Artistiek  
1             2.0        Sociaal  
2     

In [ ]:
#Add faculty to personality_result
personality_resultII = personality_result.merge(df[['dossier_id', 'faculty']], on='dossier_id', how='left')
personality_resultII = personality_resultII.drop_duplicates(subset='dossier_id', keep='first')
personality_resultII

,dossier_id,Conventioneel,Artistiek,Sociaal,Ondernemend,Realistisch,Intellectueel,pers_trait,faculty
0,1152,3.25,4.5,3.25,2.0,-1.5,1.5,Artistiek,Faculteit Geesteswetenschappen
108,1532,3.25,1.0,5.25,2.25,2.25,2.0,Sociaal,Faculteit Betawetenschappen
252,1649,-1.25,3.75,3.75,0.5,-3.0,-1.25,Artistiek,Faculteit der Diergeneeskunde
376,1663,-3.75,4.75,6.0,2.5,0.75,-1.75,Sociaal,NaN
484,2112,1.5,2.75,6.25,7.25,0.25,0.0,Ondernemend,Faculteit Geesteswetenschappen
...,...,...,...,...,...,...,...,...,...
116729,91894,5.0,2.25,1.75,1.0,-0.75,5.0,Conventioneel,Faculteit Sociale Wetenschappen
116938,91908,1.5,0.0,3.0,-0.25,1.0,0.25,Sociaal,Faculteit Geesteswetenschappen
117101,91927,1.75,0.75,2.25,-1.5,-2.5,-0.5,Sociaal,Faculteit Geowetenschappen
117280,91928,3.25,2.25,3.5,2.5,4.75,4.0,Realistisch,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
#Rename
pers_df = personality_resultII

In [ ]:
#Calculate the personality trait scores per faculty

#List of columns to sum and average
columns_to_sum = ['Conventioneel', 'Artistiek', 'Sociaal', 'Ondernemend', 'Realistisch', 'Intellectueel']

#Group by 'faculty' and calculate the sum and count for the specified columns within each faculty
grouped_df = personality_resultII.groupby('faculty')[columns_to_sum].agg(['sum', 'count'])
grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

#Print the result
print(grouped_df)



                                                  Conventioneel_sum  \
faculty                                                               
Faculteit Betawetenschappen                                   363.5   
Faculteit Geesteswetenschappen                                197.0   
Faculteit Geneeskunde                                        217.25   
Faculteit Geowetenschappen                                    204.5   
Faculteit Recht, Economie, Bestuur en Organisatie            144.75   
Faculteit Sociale Wetenschappen                               188.0   
Faculteit der Diergeneeskunde                                 76.75   
University College                                             12.0   
University College Roosevelt                                   4.25   
University College Utrecht                                    -1.25   

                                                   Conventioneel_count  \
faculty                                                                  

In [ ]:
#Adjust dataset to make it usable for visualization purposes
df = grouped_df
#Get the list of sum columns by extracting the column names that end with '_sum'
sum_columns = [col for col in df.columns if col.endswith('_sum')]

#Divide each sum column by its associated count column and rename the columns
for col in sum_columns:
    count_col = col.replace('_sum', '_count')
    avg_col = col.replace('_sum', '_avgvalue')
    df[avg_col] = df[col] / df[count_col]
    df = df.drop(columns=[col])
print(df)


                                                   Conventioneel_count  \
faculty                                                                  
Faculteit Betawetenschappen                                        138   
Faculteit Geesteswetenschappen                                     158   
Faculteit Geneeskunde                                               73   
Faculteit Geowetenschappen                                          72   
Faculteit Recht, Economie, Bestuur en Organisatie                   63   
Faculteit Sociale Wetenschappen                                    132   
Faculteit der Diergeneeskunde                                       36   
University College                                                   6   
University College Roosevelt                                         1   
University College Utrecht                                           1   

                                                   Artistiek_count  \
faculty                                  

In [ ]:
df.to_excel('new_personality_score_per_faculty.xlsx', index=True)

##OBTAIN WORKSTYLES TEST RESULTS

In [ ]:
df = finaldf

In [ ]:
#Print range of answer values for personality trait questions
min_value = df[df['maingroup'] == 'tests.workstyles.title']['answer'].min()
max_value = df[df['maingroup'] == 'tests.workstyles.title']['answer'].max()

print("Minimum value:", min_value)
print("Maximum value:", max_value)

Minimum value: 0.0
Maximum value: 1.0


In [ ]:
#Subset of the original DataFrame
subset_df = df[df['maingroup'] == 'tests.workstyles.title']

#Create the 'workstyles_result' DataFrame with desired columns
workstyles_result = pd.DataFrame(columns=['Gerichtheid op mensen','Sociale invloed','Aanpassingsvermogen',
 'Zorgvuldigheid', 'Praktisch inzicht', 'Prestatiegerichtheid',
 'Onafhankelijkheid'])

#Iterate over unique dossier_id values in the 'subset_df' and append new row to 'worksytles_result'
for dossier_id in subset_df['dossier_id'].unique():
    new_row = pd.DataFrame({'dossier_id': [dossier_id],
                            'Gerichtheid op mensen': [0],
                            'Sociale invloed': [0],
                            'Aanpassingsvermogen': [0],
                            'Zorgvuldigheid': [0],
                            'Praktisch inzicht': [0],
                            'Prestatiegerichtheid': [0],
                            'Onafhankelijkheid': [0]})


    workstyles_result = workstyles_result.append(new_row, ignore_index=True)

#Iterate over each row in the 'subset_df'
for index, row in subset_df.iterrows():
    dossier_id = row['dossier_id']
    answer = row['answer']
    level1subgroup1 = row['level1subgroup1']

    #Calculate the difference between 0.4 and the answer value
    difference = 0.5 - float(answer)

    #Check if the difference is positive or negative and update the 'workstyles_result' with the difference
    if difference > 0:
        workstyles_result.loc[workstyles_result['dossier_id'] == dossier_id, level1subgroup1] -= difference
    elif difference < 0:
        workstyles_result.loc[workstyles_result['dossier_id'] == dossier_id, level1subgroup1] += abs(difference)

print(workstyles_result)


In [ ]:
#Obtain the workstyle with the highest score for each dossierid

#List of the columns to consider
columns_to_check = ['Gerichtheid op mensen','Sociale invloed','Aanpassingsvermogen',
 'Zorgvuldigheid', 'Praktisch inzicht', 'Prestatiegerichtheid',
 'Onafhankelijkheid']

#Create a new column 'workstyle' and initialize it with None
workstyles_result['workstyle'] = None

#Iterate over each row
for index, row in workstyles_result.iterrows():
    max_value = None
    max_column = None

    #Iterate over the columns to check
    for column in columns_to_check:
        value = row[column]

        #Update the maximum value and column if necessary
        if max_value is None or value > max_value:
            max_value = value
            max_column = column

    #Assign the name of the column with the highest value to 'workstyle'
    workstyles_result.at[index, 'workstyle'] = max_column

print(workstyles_result)


    Gerichtheid op mensen Sociale invloed Aanpassingsvermogen Zorgvuldigheid  \
0                     1.3             0.1                 0.7            1.3   
1                     0.9            -0.1                 0.7            0.5   
2                     1.1             0.3                 0.1            0.1   
3                     0.3             0.3                 0.3            0.9   
4                     0.5             0.1                 0.7            0.9   
..                    ...             ...                 ...            ...   
392                   1.3             0.1                 0.9            1.3   
393                   0.5             0.1                 0.9            1.1   
394                   0.7             0.1                 0.5            1.1   
395                   0.7             0.1                 1.3            0.9   
396                   0.3            -0.1                 0.7            1.1   

    Praktisch inzicht Prestatiegerichth

In [ ]:
#Add faculty to workstyles_result
workstyles_resultII = workstyles_result.merge(df[['dossier_id', 'faculty']], on='dossier_id', how='left')
workstyles_resultII = workstyles_resultII.drop_duplicates(subset='dossier_id', keep='first')
workstyles_resultII

,Gerichtheid op mensen,Sociale invloed,Aanpassingsvermogen,Zorgvuldigheid,Praktisch inzicht,Prestatiegerichtheid,Onafhankelijkheid,dossier_id,workstyle,faculty
0,1.3,0.1,0.7,1.3,0.6,0.3,-0.1,1532.0,Gerichtheid op mensen,Faculteit Betawetenschappen
144,0.9,-0.1,0.7,0.5,0.4,-0.3,-0.1,1649.0,Gerichtheid op mensen,Faculteit der Diergeneeskunde
268,1.1,0.3,0.1,0.1,0.6,0.3,0.1,2112.0,Gerichtheid op mensen,Faculteit Geesteswetenschappen
477,0.3,0.3,0.3,0.9,0.4,0.7,-0.1,30690.0,Zorgvuldigheid,Faculteit Betawetenschappen
621,0.5,0.1,0.7,0.9,0.8,0.1,0.3,42388.0,Zorgvuldigheid,Faculteit Geowetenschappen
...,...,...,...,...,...,...,...,...,...,...
69572,1.3,0.1,0.9,1.3,0.4,0.5,0.1,91848.0,Gerichtheid op mensen,Faculteit Geesteswetenschappen
69781,0.5,0.1,0.9,1.1,0.4,0.7,0.3,91894.0,Zorgvuldigheid,Faculteit Sociale Wetenschappen
69990,0.7,0.1,0.5,1.1,0.2,0.5,0.1,91927.0,Zorgvuldigheid,Faculteit Geowetenschappen
70169,0.7,0.1,1.3,0.9,0.8,0.9,0.3,91928.0,Aanpassingsvermogen,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
workstyle_df = workstyles_resultII

In [ ]:
#Calculate the workstyle scores per faculty

#List of columns to sum and average
columns_to_sum = ['Gerichtheid op mensen','Sociale invloed','Aanpassingsvermogen',
 'Zorgvuldigheid', 'Praktisch inzicht', 'Prestatiegerichtheid',
 'Onafhankelijkheid']

#Group by 'faculty' and calculate the sum and count for the specified columns within each faculty
grouped_df = workstyles_resultII.groupby('faculty')[columns_to_sum].agg(['sum', 'count'])
grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

print(grouped_df)

                                                  Gerichtheid op mensen_sum  \
faculty                                                                       
Faculteit Betawetenschappen                                            38.9   
Faculteit Geesteswetenschappen                                         56.6   
Faculteit Geneeskunde                                                  35.0   
Faculteit Geowetenschappen                                             26.1   
Faculteit Recht, Economie, Bestuur en Organisatie                      27.2   
Faculteit Sociale Wetenschappen                                        59.8   
Faculteit der Diergeneeskunde                                           5.1   
University College                                                      0.7   
University College Roosevelt                                            1.3   
University College Utrecht                                              0.5   

                                                   

In [ ]:
df = grouped_df

#Get the list of sum columns by extracting the column names that end with '_sum'
sum_columns = [col for col in df.columns if col.endswith('_sum')]

#Divide each sum column by its associated count column and rename the columns
for col in sum_columns:
    count_col = col.replace('_sum', '_count')
    avg_col = col.replace('_sum', '_avgvalue')
    df[avg_col] = df[col] / df[count_col]
    df = df.drop(columns=[col])


print(df)

                                                   Gerichtheid op mensen_count  \
faculty                                                                          
Faculteit Betawetenschappen                                                 69   
Faculteit Geesteswetenschappen                                              84   
Faculteit Geneeskunde                                                       46   
Faculteit Geowetenschappen                                                  45   
Faculteit Recht, Economie, Bestuur en Organisatie                           36   
Faculteit Sociale Wetenschappen                                             70   
Faculteit der Diergeneeskunde                                                7   
University College                                                           1   
University College Roosevelt                                                 1   
University College Utrecht                                                   1   

               

In [ ]:
df.to_excel('new_workstyle_score_per_faculty.xlsx', index=True)

##OBTAIN WORKVALUES TEST RESULTS

In [ ]:
df = finaldf

In [ ]:
#Print range of answer values for personality trait questions
min_value = df[df['maingroup'] == 'tests.workvalues.title']['answer'].min()
max_value = df[df['maingroup'] == 'tests.workvalues.title']['answer'].max()

print("Minimum value:", min_value)
print("Maximum value:", max_value)

Minimum value: 0.0
Maximum value: 1.0


In [ ]:
#Subset of the original DataFrame
subset_df = df[df['maingroup'] == 'tests.workvalues.title']

#Create the 'workvalues_result' DataFrame with desired columns
workvalues_result = pd.DataFrame(columns=['Prestaties leveren', 'Arbeidsvoorwaarden', 'Erkenning', 'Collegialiteit',
 'Ondersteuning' ,'Onafhankelijkheid'])

#Iterate over unique dossier_id values in the 'subset_df'
for dossier_id in subset_df['dossier_id'].unique():
    #Create a new row with the dossier_id in the 'workstyles_result' DataFrame
    new_row = pd.DataFrame({'dossier_id': [dossier_id],
                            'Prestaties leveren': [0],
                            'Arbeidsvoorwaarden': [0],
                            'Erkenning': [0],
                            'Collegialiteit': [0],
                            'Ondersteuning': [0],
                            'Onafhankelijkheid': [0]})

    #Append the new row to the 'workvalues_result' DataFrame
    workvalues_result = workvalues_result.append(new_row, ignore_index=True)

#Iterate over each row in the 'subset_df'
for index, row in subset_df.iterrows():
    dossier_id = row['dossier_id']
    answer = row['answer']
    level1subgroup1 = row['level1subgroup1']

    #Calculate the difference between 0.4 and the answer value
    difference = 0.5 - float(answer)

    #Check if the difference is positive or negative and update the 'workvalues_result' using the difference
    if difference > 0:
        workvalues_result.loc[workvalues_result['dossier_id'] == dossier_id, level1subgroup1] -= difference
    elif difference < 0:
        workvalues_result.loc[workvalues_result['dossier_id'] == dossier_id, level1subgroup1] += abs(difference)

print(workvalues_result)

In [ ]:
#Obtain the workvalues with the highest score for each dossierid

#List of the columns to consider
columns_to_check = ['Prestaties leveren', 'Arbeidsvoorwaarden', 'Erkenning', 'Collegialiteit',
 'Ondersteuning' ,'Onafhankelijkheid']

#Create a new column 'workvalue' and initialize it with None
workvalues_result['workvalue'] = None

#Iterate over each row
for index, row in workvalues_result.iterrows():
    max_value = None
    max_column = None

    #Iterate over the columns to check
    for column in columns_to_check:
        value = row[column]

        #Update the maximum value and column if necessary
        if max_value is None or value > max_value:
            max_value = value
            max_column = column

    #Assign the name of the column with the highest value to 'workvalues'
    workvalues_result.at[index, 'workvalue'] = max_column

print(workvalues_result)

    Prestaties leveren Arbeidsvoorwaarden Erkenning Collegialiteit  \
0                  0.0               -1.8      -0.7            0.7   
1                  0.4               -2.0      -0.1            0.1   
2                  0.6                0.8       0.5           -0.1   
3                  0.4               -2.0      -0.7           -0.3   
4                  0.4               -1.2      -0.5           -0.9   
..                 ...                ...       ...            ...   
444                0.2               -0.2       0.1            0.5   
445                0.6                0.2      -0.3            0.7   
446                0.6                0.0      -0.1            0.3   
447                0.6                0.8       0.1            0.3   
448                0.2                1.2      -0.1            0.3   

    Ondersteuning Onafhankelijkheid  dossier_id           workvalue  
0             0.5              -0.7      1532.0      Collegialiteit  
1            -0.7  

In [ ]:
#Add faculty to workvalues_result
workvalues_resultII = workvalues_result.merge(df[['dossier_id', 'faculty']], on='dossier_id', how='left')
workvalues_resultII = workvalues_resultII.drop_duplicates(subset='dossier_id', keep='first')
workvalues_resultII

,Prestaties leveren,Arbeidsvoorwaarden,Erkenning,Collegialiteit,Ondersteuning,Onafhankelijkheid,dossier_id,workvalue,faculty
0,0.0,-1.8,-0.7,0.7,0.5,-0.7,1532.0,Collegialiteit,Faculteit Betawetenschappen
144,0.4,-2.0,-0.1,0.1,-0.7,0.3,2112.0,Prestaties leveren,Faculteit Geesteswetenschappen
353,0.6,0.8,0.5,-0.1,0.7,0.5,30690.0,Arbeidsvoorwaarden,Faculteit Betawetenschappen
497,0.4,-2.0,-0.7,-0.3,-0.3,0.9,42388.0,Onafhankelijkheid,Faculteit Geowetenschappen
641,0.4,-1.2,-0.5,-0.9,-0.1,0.3,42415.0,Prestaties leveren,Faculteit der Diergeneeskunde
...,...,...,...,...,...,...,...,...,...
76277,0.2,-0.2,0.1,0.5,0.5,0.3,91894.0,Collegialiteit,Faculteit Sociale Wetenschappen
76486,0.6,0.2,-0.3,0.7,0.3,0.1,91908.0,Collegialiteit,Faculteit Geesteswetenschappen
76649,0.6,0.0,-0.1,0.3,-0.1,0.1,91927.0,Prestaties leveren,Faculteit Geowetenschappen
76828,0.6,0.8,0.1,0.3,0.3,0.7,91928.0,Arbeidsvoorwaarden,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
workvalue_df = workvalues_resultII

In [ ]:
#Calculate the work values scores per faculty

#List of columns to sum and average
columns_to_sum = ['Prestaties leveren', 'Arbeidsvoorwaarden', 'Erkenning', 'Collegialiteit',
 'Ondersteuning' ,'Onafhankelijkheid']

#Group by 'faculty' and calculate the sum and count for the specified columns within each faculty
grouped_df = workvalues_resultII.groupby('faculty')[columns_to_sum].agg(['sum', 'count'])
grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

print(grouped_df)

                                                  Prestaties leveren_sum  \
faculty                                                                    
Faculteit Betawetenschappen                                         30.2   
Faculteit Geesteswetenschappen                                      34.4   
Faculteit Geneeskunde                                               17.4   
Faculteit Geowetenschappen                                          18.6   
Faculteit Recht, Economie, Bestuur en Organisatie                   15.6   
Faculteit Sociale Wetenschappen                                     34.0   
Faculteit der Diergeneeskunde                                        7.6   
University College                                                   1.4   
University College Roosevelt                                         0.6   
University College Utrecht                                           0.4   

                                                   Prestaties leveren_count  \
faculty 

In [ ]:
df = grouped_df

#Get the list of sum columns by extracting the column names that end with '_sum'
sum_columns = [col for col in df.columns if col.endswith('_sum')]

#Divide each sum column by its associated count column and rename the columns
for col in sum_columns:
    count_col = col.replace('_sum', '_count')
    avg_col = col.replace('_sum', '_avgvalue')
    df[avg_col] = df[col] / df[count_col]
    df = df.drop(columns=[col])

print(df)

In [ ]:
df.to_excel('new_workvalue_score_per_faculty.xlsx', index=True)

##OBTAIN COMPETENCES TEST RESULTS (grouped using subgroup headers)

In [ ]:
df = finaldf
print(df.head(5))

   question_id  dossier_id answeredTest_id          question  \
0         2281        1152           19833  question.id_2281   
1         2263        1152           19833  question.id_2263   
2         2269        1152           19833  question.id_2269   
3         2278        1152           19833  question.id_2278   
4         2280        1152           19833  question.id_2280   

                                            type  \
0  likert/likert5puntsacceptatie2positiefpluseen   
1  likert/likert5puntsacceptatie2positiefpluseen   
2  likert/likert5puntsacceptatie2positiefpluseen   
3  likert/likert5puntsacceptatie2positiefpluseen   
4  likert/likert5puntsacceptatie2positiefpluseen   

                                     label            datetime  \
0                      rapporten schrijven 2022-06-27 12:36:27   
1                           intuïtief zijn 2022-06-27 12:36:27   
2             je gevoelens duidelijk uiten 2022-06-27 12:36:27   
3  activiteiten en evenementen organis

In [ ]:
com_df = pd.read_excel("competences_labels.xlsx")
print(com_df.head(5))

     id          question                                   type  \
0  2535  question.id_2535  likert/likert7puntslegeschaalpositief   
1  2536  question.id_2536  likert/likert7puntslegeschaalpositief   
2  2537  question.id_2537  likert/likert7puntslegeschaalpositief   
3  2538  question.id_2538  likert/likert7puntslegeschaalpositief   
4  2539  question.id_2539  likert/likert7puntslegeschaalpositief   

                    description           createdOn        lastModified  \
0  question.id_2535_description 2021-03-19 10:25:05 2022-07-20 20:36:08   
1  question.id_2536_description 2021-03-19 10:47:57 2022-07-20 21:08:12   
2  question.id_2537_description 2021-03-19 10:57:31 2022-07-20 21:05:17   
3  question.id_2538_description 2021-03-19 11:01:31 2022-07-20 20:55:47   
4  question.id_2539_description 2021-03-19 14:56:49 2022-07-20 20:56:00   

  knowledgeBaseKey        subcompetences maincompetences maincompetencesII  
0               \N  readingComprehension         content       

In [ ]:
#Add maincompetences to the df
df = pd.merge(df, com_df[['question', 'maincompetences']], on='question', how='left')
df

,question_id,dossier_id,answeredTest_id,question,type,label,datetime,maingroup,maingroup_id,level1subgroup1,faculty,answer,maincompetences
0,2281,1152,19833,question.id_2281,likert/likert5puntsacceptatie2positiefpluseen,rapporten schrijven,2022-06-27 12:36:27,Beroepskeuzetest,459.0,Conventioneel,Faculteit Geesteswetenschappen,0.75,NaN
1,2263,1152,19833,question.id_2263,likert/likert5puntsacceptatie2positiefpluseen,intuïtief zijn,2022-06-27 12:36:27,Beroepskeuzetest,459.0,Artistiek,Faculteit Geesteswetenschappen,0.75,NaN
2,2269,1152,19833,question.id_2269,likert/likert5puntsacceptatie2positiefpluseen,je gevoelens duidelijk uiten,2022-06-27 12:36:27,Beroepskeuzetest,459.0,Sociaal,Faculteit Geesteswetenschappen,0.75,NaN
3,2278,1152,19833,question.id_2278,likert/likert5puntsacceptatie2positiefpluseen,activiteiten en evenementen organiseren,2022-06-27 12:36:27,Beroepskeuzetest,459.0,Ondernemend,Faculteit Geesteswetenschappen,0.75,NaN
4,2280,1152,19833,question.id_2280,likert/likert5puntsacceptatie2positiefpluseen,onderdeel zijn van een organisatie,2022-06-27 12:36:27,Beroepskeuzetest,459.0,Conventioneel,Faculteit Geesteswetenschappen,0.75,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117734,2411,91930,25252,question.id_2411,likert/likert6puntsmatepositief,Om mijn werk goed te doen heb je creativiteit ...,2023-04-27 21:06:14,tests.workstyles.title,516.0,Praktisch inzicht,Faculteit Geesteswetenschappen,0.80,NaN
117735,2397,91930,25252,question.id_2397,likert/likert6puntsmatepositief,Om mijn werk goed te doen is het belangrijk om...,2023-04-27 21:06:14,tests.workstyles.title,516.0,Prestatiegerichtheid,Faculteit Geesteswetenschappen,0.40,NaN
117736,2398,91930,25252,question.id_2398,likert/likert6puntsmatepositief,Om mijn werk goed te doen is doorzettingsvermo...,2023-04-27 21:06:14,tests.workstyles.title,516.0,Prestatiegerichtheid,Faculteit Geesteswetenschappen,0.60,NaN
117737,2399,91930,25252,question.id_2399,likert/likert6puntsmatepositief,Om mijn werk goed te doen is het belangrijk om...,2023-04-27 21:06:14,tests.workstyles.title,516.0,Prestatiegerichtheid,Faculteit Geesteswetenschappen,0.60,NaN


In [ ]:
#Check whether merge was successful
print(df['maincompetences'].unique())

[nan 'content' 'process' 'socialskills' 'complexproblemsolving'
 'technicalskills' 'systemskills' 'resourcemanagementskills']


In [ ]:
#Filter rows where 'maincompetences' is not NaN
filtered_df = df.dropna(subset=['maincompetences'])

#Select the 'answer' column from the filtered DataFrame
answer_column = filtered_df['answer']

#Calculate the range of values in the 'answer' column
answer_range = answer_column.min(), answer_column.max()

print("Range of values in 'answer' column when 'maincompetences' is not NaN:")
print(answer_range)

Range of values in 'answer' column when 'maincompetences' is not NaN:
(0.0, 1.0)


In [ ]:
#Subset of the original DataFrame
subset_df = filtered_df

#Create the 'competences_result' DataFrame with desired columns
competences_result = pd.DataFrame(columns=['content', 'process', 'socialskills', 'complexproblemsolving',
 'technicalskills' ,'systemskills', 'resourcemanagementskills'])

#Iterate over unique dossier_id values in the 'subset_df' and append the new row to 'competences_result'
for dossier_id in subset_df['dossier_id'].unique():
    new_row = pd.DataFrame({'dossier_id': [dossier_id],
                            'content': [0],
                            'process': [0],
                            'socialskills': [0],
                            'complexproblemsolving': [0],
                            'technicalskills': [0],
                            'systemskills': [0],
                            'resourcemanagementskills': [0]})
    competences_result = competences_result.append(new_row, ignore_index=True)

#Iterate over each row in the 'subset_df'
for index, row in subset_df.iterrows():
    dossier_id = row['dossier_id']
    answer = row['answer']
    maincompetences = row['maincompetences']

    #Calculate the difference between 0.4 and the answer value
    difference = 0.5 - float(answer)

    #Check if the difference is positive or negative and update the 'competences_result' with the difference
    if difference > 0:
        competences_result.loc[competences_result['dossier_id'] == dossier_id, maincompetences] -= difference
    elif difference < 0:
        competences_result.loc[competences_result['dossier_id'] == dossier_id, maincompetences] += abs(difference)

print(competences_result)


In [ ]:
#Obtain the competences with the highest score for each dossierid

#List of the columns to consider
columns_to_check = ['content', 'process', 'socialskills', 'complexproblemsolving',
 'technicalskills' ,'systemskills', 'resourcemanagementskills']

#Create a new column 'competences' and initialize it with None
competences_result['competences'] = None

#Iterate over each row
for index, row in competences_result.iterrows():
    max_value = None
    max_column = None

    #Iterate over the columns to check
    for column in columns_to_check:
        value = row[column]

        #Update the maximum value and column if necessary
        if max_value is None or value > max_value:
            max_value = value
            max_column = column

    #Assign the name of the column with the highest value to 'competences'
    competences_result.at[index, 'competences'] = max_column

print(competences_result)

      content   process socialskills complexproblemsolving technicalskills  \
0         0.0       1.0          0.5                     0       -2.666667   
1    0.333333      -0.5         -0.5             -0.166667            -0.5   
2    0.833333  0.333333         -0.0             -0.166667       -0.166667   
3    0.333333  0.666667     1.166667              0.166667            -0.0   
4    0.166667       1.0         -1.0              0.166667            -1.5   
..        ...       ...          ...                   ...             ...   
431       0.5  0.666667         -0.0              0.166667       -0.166667   
432       0.5  0.333333     0.166667              0.166667       -1.833333   
433 -0.333333      -0.0         -0.5                     0       -3.333333   
434  1.333333  0.666667     0.833333              0.333333             2.0   
435      -0.5      -0.0    -0.666667                     0            -2.5   

    systemskills resourcemanagementskills  dossier_id          

In [ ]:
#Add faculty to competences_result
competences_resultII = competences_result.merge(df[['dossier_id', 'faculty']], on='dossier_id', how='left')
competences_resultII = competences_resultII.drop_duplicates(subset='dossier_id', keep='first')
competences_resultII

,content,process,socialskills,complexproblemsolving,technicalskills,systemskills,resourcemanagementskills,dossier_id,competences,faculty
0,0.0,1.0,0.5,0,-2.666667,-0.5,0,2112.0,process,Faculteit Geesteswetenschappen
209,0.333333,-0.5,-0.5,-0.166667,-0.5,-0.166667,-0.0,42916.0,content,Faculteit der Diergeneeskunde
388,0.833333,0.333333,-0.0,-0.166667,-0.166667,-0.333333,-0.5,47861.0,content,Faculteit Geowetenschappen
547,0.333333,0.666667,1.166667,0.166667,-0.0,0.166667,1.166667,50367.0,socialskills,Faculteit Geowetenschappen
725,0.166667,1.0,-1.0,0.166667,-1.5,-0.5,-1.0,55937.0,process,Faculteit der Diergeneeskunde
...,...,...,...,...,...,...,...,...,...,...
76031,0.5,0.666667,-0.0,0.166667,-0.166667,0,0.166667,91894.0,process,Faculteit Sociale Wetenschappen
76240,0.5,0.333333,0.166667,0.166667,-1.833333,0,-0.833333,91908.0,content,Faculteit Geesteswetenschappen
76403,-0.333333,-0.0,-0.5,0,-3.333333,-0.666667,-1.0,91927.0,complexproblemsolving,Faculteit Geowetenschappen
76582,1.333333,0.666667,0.833333,0.333333,2.0,0.5,0.166667,91928.0,technicalskills,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
comp_df = competences_resultII

In [ ]:
#List of columns to sum and average
columns_to_sum = ['content', 'process', 'socialskills', 'complexproblemsolving',
 'technicalskills' ,'systemskills', 'resourcemanagementskills']

#Group by 'faculty' and calculate the sum and count for the specified columns within each faculty
grouped_df = competences_resultII.groupby('faculty')[columns_to_sum].agg(['sum', 'count'])
grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

print(grouped_df)

                                                  content_sum  content_count  \
faculty                                                                        
Faculteit Betawetenschappen                              49.0             78   
Faculteit Geesteswetenschappen                      39.333333             98   
Faculteit Geneeskunde                               30.166667             45   
Faculteit Geowetenschappen                               38.0             48   
Faculteit Recht, Economie, Bestuur en Organisatie        16.0             39   
Faculteit Sociale Wetenschappen                     27.833333             79   
Faculteit der Diergeneeskunde                        1.666667              3   
University College                                   1.833333              3   
University College Utrecht                                1.0              1   

                                                  process_sum  process_count  \
faculty                                

In [ ]:
df = grouped_df
#Get the list of sum columns by extracting the column names that end with '_sum'
sum_columns = [col for col in df.columns if col.endswith('_sum')]

#Divide each sum column by its associated count column and rename the columns
for col in sum_columns:
    count_col = col.replace('_sum', '_count')
    avg_col = col.replace('_sum', '_avgvalue')
    df[avg_col] = df[col] / df[count_col]
    df = df.drop(columns=[col])

print(df)

                                                   content_count  \
faculty                                                            
Faculteit Betawetenschappen                                   78   
Faculteit Geesteswetenschappen                                98   
Faculteit Geneeskunde                                         45   
Faculteit Geowetenschappen                                    48   
Faculteit Recht, Economie, Bestuur en Organisatie             39   
Faculteit Sociale Wetenschappen                               79   
Faculteit der Diergeneeskunde                                  3   
University College                                             3   
University College Utrecht                                     1   

                                                   process_count  \
faculty                                                            
Faculteit Betawetenschappen                                   78   
Faculteit Geesteswetenschappen                 

In [ ]:
df.to_excel('new_competences_score_per_faculty.xlsx', index=True)

##MERGE ALL TESTRESULTS

In [ ]:
#Modify 'dossier_id' column in comp_df
comp_df['dossier_id'] = comp_df['dossier_id'].astype(str).str.rstrip('.0').astype(int)

#Convert 'dossier_id' column to string and remove trailing ",0" in workstyle_df
workstyle_df['dossier_id'] = workstyle_df['dossier_id'].astype(str).str.rstrip('.0').astype(int)

#Convert 'dossier_id' column to string and remove trailing ",0" in workvalue_df
workvalue_df['dossier_id'] = workvalue_df['dossier_id'].astype(str).str.rstrip('.0').astype(int)


In [ ]:
comp_df

,content,process,socialskills,complexproblemsolving,technicalskills,systemskills,resourcemanagementskills,dossier_id,competences,faculty
0,0.0,1.0,0.5,0,-2.666667,-0.5,0,2112,process,Faculteit Geesteswetenschappen
209,0.333333,-0.5,-0.5,-0.166667,-0.5,-0.166667,-0.0,42916,content,Faculteit der Diergeneeskunde
388,0.833333,0.333333,-0.0,-0.166667,-0.166667,-0.333333,-0.5,47861,content,Faculteit Geowetenschappen
547,0.333333,0.666667,1.166667,0.166667,-0.0,0.166667,1.166667,50367,socialskills,Faculteit Geowetenschappen
725,0.166667,1.0,-1.0,0.166667,-1.5,-0.5,-1.0,55937,process,Faculteit der Diergeneeskunde
...,...,...,...,...,...,...,...,...,...,...
76031,0.5,0.666667,-0.0,0.166667,-0.166667,0,0.166667,91894,process,Faculteit Sociale Wetenschappen
76240,0.5,0.333333,0.166667,0.166667,-1.833333,0,-0.833333,91908,content,Faculteit Geesteswetenschappen
76403,-0.333333,-0.0,-0.5,0,-3.333333,-0.666667,-1.0,91927,complexproblemsolving,Faculteit Geowetenschappen
76582,1.333333,0.666667,0.833333,0.333333,2.0,0.5,0.166667,91928,technicalskills,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
workstyle_df

,Gerichtheid op mensen,Sociale invloed,Aanpassingsvermogen,Zorgvuldigheid,Praktisch inzicht,Prestatiegerichtheid,Onafhankelijkheid,dossier_id,workstyle,faculty
0,1.3,0.1,0.7,1.3,0.6,0.3,-0.1,1532,Gerichtheid op mensen,Faculteit Betawetenschappen
144,0.9,-0.1,0.7,0.5,0.4,-0.3,-0.1,1649,Gerichtheid op mensen,Faculteit der Diergeneeskunde
268,1.1,0.3,0.1,0.1,0.6,0.3,0.1,2112,Gerichtheid op mensen,Faculteit Geesteswetenschappen
477,0.3,0.3,0.3,0.9,0.4,0.7,-0.1,3069,Zorgvuldigheid,Faculteit Betawetenschappen
621,0.5,0.1,0.7,0.9,0.8,0.1,0.3,42388,Zorgvuldigheid,Faculteit Geowetenschappen
...,...,...,...,...,...,...,...,...,...,...
69572,1.3,0.1,0.9,1.3,0.4,0.5,0.1,91848,Gerichtheid op mensen,Faculteit Geesteswetenschappen
69781,0.5,0.1,0.9,1.1,0.4,0.7,0.3,91894,Zorgvuldigheid,Faculteit Sociale Wetenschappen
69990,0.7,0.1,0.5,1.1,0.2,0.5,0.1,91927,Zorgvuldigheid,Faculteit Geowetenschappen
70169,0.7,0.1,1.3,0.9,0.8,0.9,0.3,91928,Aanpassingsvermogen,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
workvalue_df

,Prestaties leveren,Arbeidsvoorwaarden,Erkenning,Collegialiteit,Ondersteuning,Onafhankelijkheid,dossier_id,workvalue,faculty
0,0.0,-1.8,-0.7,0.7,0.5,-0.7,1532,Collegialiteit,Faculteit Betawetenschappen
144,0.4,-2.0,-0.1,0.1,-0.7,0.3,2112,Prestaties leveren,Faculteit Geesteswetenschappen
353,0.6,0.8,0.5,-0.1,0.7,0.5,3069,Arbeidsvoorwaarden,Faculteit Betawetenschappen
497,0.4,-2.0,-0.7,-0.3,-0.3,0.9,42388,Onafhankelijkheid,Faculteit Geowetenschappen
641,0.4,-1.2,-0.5,-0.9,-0.1,0.3,42415,Prestaties leveren,Faculteit der Diergeneeskunde
...,...,...,...,...,...,...,...,...,...
76277,0.2,-0.2,0.1,0.5,0.5,0.3,91894,Collegialiteit,Faculteit Sociale Wetenschappen
76486,0.6,0.2,-0.3,0.7,0.3,0.1,91908,Collegialiteit,Faculteit Geesteswetenschappen
76649,0.6,0.0,-0.1,0.3,-0.1,0.1,91927,Prestaties leveren,Faculteit Geowetenschappen
76828,0.6,0.8,0.1,0.3,0.3,0.7,91928,Arbeidsvoorwaarden,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
pers_df

,dossier_id,Conventioneel,Artistiek,Sociaal,Ondernemend,Realistisch,Intellectueel,pers_trait,faculty
0,1152,3.25,4.5,3.25,2.0,-1.5,1.5,Artistiek,Faculteit Geesteswetenschappen
108,1532,3.25,1.0,5.25,2.25,2.25,2.0,Sociaal,Faculteit Betawetenschappen
252,1649,-1.25,3.75,3.75,0.5,-3.0,-1.25,Artistiek,Faculteit der Diergeneeskunde
376,1663,-3.75,4.75,6.0,2.5,0.75,-1.75,Sociaal,NaN
484,2112,1.5,2.75,6.25,7.25,0.25,0.0,Ondernemend,Faculteit Geesteswetenschappen
...,...,...,...,...,...,...,...,...,...
116729,91894,5.0,2.25,1.75,1.0,-0.75,5.0,Conventioneel,Faculteit Sociale Wetenschappen
116938,91908,1.5,0.0,3.0,-0.25,1.0,0.25,Sociaal,Faculteit Geesteswetenschappen
117101,91927,1.75,0.75,2.25,-1.5,-2.5,-0.5,Sociaal,Faculteit Geowetenschappen
117280,91928,3.25,2.25,3.5,2.5,4.75,4.0,Realistisch,"Faculteit Recht, Economie, Bestuur en Organisatie"


In [ ]:
#Merge DataFrames based on 'dossier_id'
merged_df = pd.merge(comp_df, workstyle_df, on='dossier_id', how='outer')
merged_df.columns


Index(['content', 'process', 'socialskills', 'complexproblemsolving',
       'technicalskills', 'systemskills', 'resourcemanagementskills',
       'dossier_id', 'competences', 'faculty_x', 'Gerichtheid op mensen',
       'Sociale invloed', 'Aanpassingsvermogen', 'Zorgvuldigheid',
       'Praktisch inzicht', 'Prestatiegerichtheid', 'Onafhankelijkheid',
       'workstyle', 'faculty_y'],
      dtype='object')

In [ ]:
merged_df['faculty'] = merged_df['faculty_x'].combine_first(merged_df['faculty_x']).fillna(merged_df['faculty_y'])
merged_df = merged_df.drop(['faculty_x', 'faculty_y'], axis=1)

In [ ]:
merged_df = pd.merge(merged_df, pers_df, on='dossier_id', how='outer')
merged_df.columns

<ipython-input-101-95965b14ae7f>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  merged_df = pd.merge(merged_df, pers_df, on='dossier_id', how='outer')


Index(['content', 'process', 'socialskills', 'complexproblemsolving',
       'technicalskills', 'systemskills', 'resourcemanagementskills',
       'dossier_id', 'competences', 'Gerichtheid op mensen', 'Sociale invloed',
       'Aanpassingsvermogen', 'Zorgvuldigheid', 'Praktisch inzicht',
       'Prestatiegerichtheid', 'Onafhankelijkheid', 'workstyle', 'faculty_x',
       'Conventioneel', 'Artistiek', 'Sociaal', 'Ondernemend', 'Realistisch',
       'Intellectueel', 'pers_trait', 'faculty_y'],
      dtype='object')

In [ ]:
merged_df['faculty'] = merged_df['faculty_x'].combine_first(merged_df['faculty_x']).fillna(merged_df['faculty_y'])
merged_df = merged_df.drop(['faculty_x', 'faculty_y'], axis=1)

In [ ]:
merged_df = pd.merge(merged_df, workvalue_df, on='dossier_id', how='outer')

<ipython-input-103-58548a6f7a27>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  merged_df = pd.merge(merged_df, workvalue_df, on='dossier_id', how='outer')


In [ ]:
merged_df['faculty'] = merged_df['faculty_x'].combine_first(merged_df['faculty_x']).fillna(merged_df['faculty_y'])
merged_df = merged_df.drop(['faculty_x', 'faculty_y'], axis=1)

In [ ]:
merged_df

,content,process,socialskills,complexproblemsolving,technicalskills,systemskills,resourcemanagementskills,dossier_id,competences,Gerichtheid op mensen,...,Intellectueel,pers_trait,Prestaties leveren,Arbeidsvoorwaarden,Erkenning,Collegialiteit,Ondersteuning,Onafhankelijkheid_y,workvalue,faculty
0,0.0,1.0,0.5,0,-2.666667,-0.5,0,2112.0,process,1.1,...,0.0,Ondernemend,0.4,-2.0,-0.1,0.1,-0.7,0.3,Prestaties leveren,Faculteit Geesteswetenschappen
1,0.333333,-0.5,-0.5,-0.166667,-0.5,-0.166667,-0.0,42916.0,content,0.7,...,3.75,Realistisch,0.6,1.0,-0.1,0.7,0.9,0.5,Arbeidsvoorwaarden,Faculteit der Diergeneeskunde
2,0.833333,0.333333,-0.0,-0.166667,-0.166667,-0.333333,-0.5,47861.0,content,0.7,...,3.0,Realistisch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Faculteit Geowetenschappen
3,0.333333,0.666667,1.166667,0.166667,-0.0,0.166667,1.166667,50367.0,socialskills,0.7,...,4.75,Intellectueel,0.2,0.0,0.3,0.7,0.5,0.1,Collegialiteit,Faculteit Geowetenschappen
4,0.166667,1.0,-1.0,0.166667,-1.5,-0.5,-1.0,55937.0,process,0.5,...,5.5,Realistisch,0.4,-1.6,-1.3,0.3,0.5,-0.3,Ondersteuning,Faculteit der Diergeneeskunde
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8207.0,NaN,NaN,...,NaN,NaN,0.6,-1.0,-0.9,-0.3,0.3,-0.7,Prestaties leveren,NaN
806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8234.0,NaN,NaN,...,NaN,NaN,0.2,-1.4,-1.1,-0.3,0.3,0.3,Ondersteuning,Faculteit Geneeskunde
807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8296.0,NaN,NaN,...,NaN,NaN,0.2,-1.4,-0.5,-0.3,0.5,-0.5,Ondersteuning,Faculteit Betawetenschappen
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8352.0,NaN,NaN,...,NaN,NaN,0.4,-0.6,-1.3,0.3,0.1,-0.9,Prestaties leveren,Faculteit Betawetenschappen


In [ ]:
print(merged_df.columns)

Index(['content', 'process', 'socialskills', 'complexproblemsolving',
       'technicalskills', 'systemskills', 'resourcemanagementskills',
       'dossier_id', 'competences', 'Gerichtheid op mensen', 'Sociale invloed',
       'Aanpassingsvermogen', 'Zorgvuldigheid', 'Praktisch inzicht',
       'Prestatiegerichtheid', 'Onafhankelijkheid_x', 'workstyle',
       'Conventioneel', 'Artistiek', 'Sociaal', 'Ondernemend', 'Realistisch',
       'Intellectueel', 'pers_trait', 'Prestaties leveren',
       'Arbeidsvoorwaarden', 'Erkenning', 'Collegialiteit', 'Ondersteuning',
       'Onafhankelijkheid_y', 'workvalue', 'faculty'],
      dtype='object')


In [ ]:
#Change Onafhankelijkheid_x to Onafhankelijkheid_ws (workstyle) and Onafhankelijkheid_y to Onafhankelijkheid_wv (workvalue)
merged_df = merged_df.rename(columns={'Onafhankelijkheid_x': 'Onafhankelijkheid_ws', 'Onafhankelijkheid_y': 'Onafhankelijkheid_wv'})


In [ ]:
merged_df

,content,process,socialskills,complexproblemsolving,technicalskills,systemskills,resourcemanagementskills,dossier_id,competences,Gerichtheid op mensen,...,Intellectueel,pers_trait,Prestaties leveren,Arbeidsvoorwaarden,Erkenning,Collegialiteit,Ondersteuning,Onafhankelijkheid_wv,workvalue,faculty
0,0.0,1.0,0.5,0,-2.666667,-0.5,0,2112.0,process,1.1,...,0.0,Ondernemend,0.4,-2.0,-0.1,0.1,-0.7,0.3,Prestaties leveren,Faculteit Geesteswetenschappen
1,0.333333,-0.5,-0.5,-0.166667,-0.5,-0.166667,-0.0,42916.0,content,0.7,...,3.75,Realistisch,0.6,1.0,-0.1,0.7,0.9,0.5,Arbeidsvoorwaarden,Faculteit der Diergeneeskunde
2,0.833333,0.333333,-0.0,-0.166667,-0.166667,-0.333333,-0.5,47861.0,content,0.7,...,3.0,Realistisch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Faculteit Geowetenschappen
3,0.333333,0.666667,1.166667,0.166667,-0.0,0.166667,1.166667,50367.0,socialskills,0.7,...,4.75,Intellectueel,0.2,0.0,0.3,0.7,0.5,0.1,Collegialiteit,Faculteit Geowetenschappen
4,0.166667,1.0,-1.0,0.166667,-1.5,-0.5,-1.0,55937.0,process,0.5,...,5.5,Realistisch,0.4,-1.6,-1.3,0.3,0.5,-0.3,Ondersteuning,Faculteit der Diergeneeskunde
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8207.0,NaN,NaN,...,NaN,NaN,0.6,-1.0,-0.9,-0.3,0.3,-0.7,Prestaties leveren,NaN
806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8234.0,NaN,NaN,...,NaN,NaN,0.2,-1.4,-1.1,-0.3,0.3,0.3,Ondersteuning,Faculteit Geneeskunde
807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8296.0,NaN,NaN,...,NaN,NaN,0.2,-1.4,-0.5,-0.3,0.5,-0.5,Ondersteuning,Faculteit Betawetenschappen
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8352.0,NaN,NaN,...,NaN,NaN,0.4,-0.6,-1.3,0.3,0.1,-0.9,Prestaties leveren,Faculteit Betawetenschappen


In [ ]:
#Count the number of missing values per column
missing_values_count = merged_df.isna().sum()

#Print the result
print(missing_values_count)

content                     374
process                     374
socialskills                374
complexproblemsolving       374
technicalskills             374
systemskills                374
resourcemanagementskills    374
dossier_id                    0
competences                 374
Gerichtheid op mensen       413
Sociale invloed             413
Aanpassingsvermogen         413
Zorgvuldigheid              413
Praktisch inzicht           413
Prestatiegerichtheid        413
Onafhankelijkheid_ws        413
workstyle                   413
Conventioneel                61
Artistiek                    61
Sociaal                      61
Ondernemend                  61
Realistisch                  61
Intellectueel                61
pers_trait                   61
Prestaties leveren          361
Arbeidsvoorwaarden          361
Erkenning                   361
Collegialiteit              361
Ondersteuning               361
Onafhankelijkheid_wv        361
workvalue                   361
faculty 

In [ ]:
merged_df.to_excel('full_results_df.xlsx', index=True)